In [1]:
import pandas as pd
import numpy as np
import pickle
import itertools
import gc
import math
import matplotlib.pyplot as plt
import dateutil.easter as easter
from matplotlib.ticker import MaxNLocator, FormatStrFormatter, PercentFormatter
from datetime import datetime, date, timedelta
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_percentage_error
import scipy.stats

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
NO_STORE = True

original_train_df = pd.read_csv('/home/onyxia/work/Forecasting_Sticker_Sales/train.csv')
original_test_df = pd.read_csv('/home/onyxia/work/Forecasting_Sticker_Sales/test.csv')

gdp_df = pd.read_csv('/home/onyxia/work/Forecasting_Sticker_Sales/linear_model/filtered_gdp_per_capita.csv')
gdp_df.set_index('year', inplace=True)

cci_df = pd.read_csv('/home/onyxia/work/Forecasting_Sticker_Sales/linear_model/DP_LIVE_21012022073653464.csv')
cci_df.set_index(['LOCATION', 'TIME'], inplace=True)

# The dates are read as strings and must be converted
for df in [original_train_df, original_test_df]:
    df['date'] = pd.to_datetime(df.date)
original_train_df.head(6)

,id,date,country,store,product,num_sold
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
1,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
3,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
4,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0
5,5,2010-01-01,Canada,Stickers for Less,Holographic Goose,300.0


In [4]:
display(original_train_df.head())
display(gdp_df.head())
display(cci_df.head())

,id,date,country,store,product,num_sold
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
1,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
3,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
4,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0


,GDP_Canada,GDP_Finland,GDP_Italy,GDP_Kenya,GDP_Norway,GDP_Singapore
year,,,,,,
2010,47562.083425,46459.973255,36000.520118,1080.296184,87693.790066,47236.960235
2011,52223.696112,51081.997670,38599.062207,1085.487152,100600.562408,53890.428727
2012,52669.089963,47710.790217,35053.526244,1271.815383,101524.141852,55546.488539
2013,52635.174958,49878.043244,35549.974697,1354.820833,102913.450844,56967.425794
2014,50955.998323,50260.299859,35518.415292,1462.220052,97019.182753,57562.530794


INDICATOR   SUBJECT    MEASURE FREQUENCY     Value  \
LOCATION TIME                                                         
NLD      1973-01       CCI  AMPLITUD  LTRENDIDX         M  101.5028   
         1973-02       CCI  AMPLITUD  LTRENDIDX         M  101.4815   
         1973-03       CCI  AMPLITUD  LTRENDIDX         M  101.3081   
         1973-04       CCI  AMPLITUD  LTRENDIDX         M  101.0173   
         1973-05       CCI  AMPLITUD  LTRENDIDX         M  100.8456   

                  Flag Codes  
LOCATION TIME                 
NLD      1973-01         NaN  
         1973-02         NaN  
         1973-03         NaN  
         1973-04         NaN  
         1973-05         NaN

In [5]:
unique_countries = cci_df.index.get_level_values(0).unique()
print(f"Number of countries: {len(unique_countries)}")
print("Countries:", unique_countries.tolist())

Number of countries: 44
Countries: ['NLD', 'CHE', 'FRA', 'POL', 'CZE', 'JPN', 'OECDE', 'AUS', 'OECD', 'SWE', 'MEX', 'GBR', 'ZAF', 'USA', 'HUN', 'PRT', 'DNK', 'ESP', 'LUX', 'GRC', 'BRA', 'SVK', 'CHN', 'BEL', 'FIN', 'NZL', 'G-7', 'IDN', 'TUR', 'AUT', 'ITA', 'IRL', 'SVN', 'DEU', 'KOR', 'EST', 'EA19', 'ISR', 'RUS', 'LVA', 'LTU', 'COL', 'CHL', 'CRI']


In [6]:
def mape(y_true, y_pred):
    """
    MAPE Loss

    """
    mape = mean_absolute_percentage_error(actual, predicted) * 100
    return mape

### Feature Engineering

In [7]:
cci_df.columns

Index(['INDICATOR', 'SUBJECT', 'MEASURE', 'FREQUENCY', 'Value', 'Flag Codes'], dtype='object')

In [8]:
print(cci_df['INDICATOR'].unique())
print(cci_df['SUBJECT'].unique())
print(cci_df['MEASURE'].unique())
print(cci_df['FREQUENCY'].unique())
print(cci_df['Value'].unique())
print(cci_df['Flag Codes'].unique())

['CCI']
['AMPLITUD']
['LTRENDIDX']
['M']
[101.5028  101.4815  101.3081  ...  99.4906   99.57128  99.61041]
[nan]


In [9]:
# List of countries to check
countries = ['Canada', 'Finland', 'Italy', 'Kenya', 'Norway', 'Singapore']

# Mapping country names to ISO Alpha-3 codes
country_codes = {
    'Canada': 'CAN',
    'Finland': 'FIN',
    'Italy': 'ITA',
    'Kenya': 'KEN',
    'Norway': 'NOR',
    'Singapore': 'SGP'
}

# Extracting the 'LOCATION' index
locations_in_index = cci_df.index.get_level_values('LOCATION')

# Check for existence of countries
existing_countries = [country for country in countries if country_codes.get(country, country) in locations_in_index]
missing_countries = [country for country in countries if country_codes.get(country, country) not in locations_in_index]

print("Existing countries:", existing_countries)
print("Missing countries:", missing_countries)

Existing countries: ['Finland', 'Italy']
Missing countries: ['Canada', 'Kenya', 'Norway', 'Singapore']


In [10]:
def engineer(df):
    """Return a new dataframe with the engineered features"""
    
    def get_gdp(row):
        country = 'GDP_' + row.country
        return gdp_df.loc[row.date.year, country]

#    def get_cci(row):
#       country = row.country
#       time = f"{row.date.year}-{row.date.month:02d}"
#       return cci_df.loc[country[:3].upper(), time].Value

    new_df = pd.DataFrame({'gdp': np.log(df.apply(get_gdp, axis=1)),
#                           'cci': df.apply(get_cci, axis=1),
                           'wd4': df.date.dt.weekday == 4, # Friday
                           'wd56': df.date.dt.weekday >= 5, # Saturday and Sunday
                          })

    # One-hot encoding (no need to encode the last categories)
    for country in ['Canada', 'Finland', 'Italy', 'Kenya', 'Norway']:
        new_df[country] = df.country == country
    for store in ['Discount Stickers', 'Stickers for Less']:
        new_df[store] = df['store'] == store
    for product in ['Holographic Goose', 'Kaggle', 'Kaggle Tiers', 'Kerneler']:
        new_df[product] = df['product'] == product

    # Seasonal variations (Fourier series)
    dayofyear = df.date.dt.dayofyear

    for k in range(1, 3):  # For harmonics k = 1, 2
        # Generate the sine and cosine terms for the seasonal variation
        sink = np.sin(dayofyear / 365 * 2 * math.pi * k)
        cosk = np.cos(dayofyear / 365 * 2 * math.pi * k)

        new_df[f'HG_sin{k}'] = sink * new_df['Holographic Goose']
        new_df[f'HG_cos{k}'] = cosk * new_df['Holographic Goose']
        new_df[f'Kaggle_sin{k}'] = sink * new_df['Kaggle']
        new_df[f'Kaggle_cos{k}'] = cosk * new_df['Kaggle']
        new_df[f'KT_sin{k}'] = sink * new_df['Kaggle Tiers']
        new_df[f'KT_cos{k}'] = cosk * new_df['Kaggle Tiers']
        new_df[f'Kerneler_sin{k}'] = sink * new_df['Kerneler']
        new_df[f'Kerneler_cos{k}'] = cosk * new_df['Kerneler']

    # Special days
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"dec{d}":
                                      (df.date.dt.month == 12) & (df.date.dt.day == d)
                                      for d in range(24, 32)}),
                        pd.DataFrame({f"n-dec{d}":
                                      (df.date.dt.month == 12) & (df.date.dt.day == d) & (df.country == 'Norway')
                                      for d in range(25, 32)}),
                        pd.DataFrame({f"f-jan{d}":
                                      (df.date.dt.month == 1) & (df.date.dt.day == d) & (df.country == 'Finland')
                                      for d in range(1, 15)}),
                        pd.DataFrame({f"n-jan{d}":
                                      (df.date.dt.month == 1) & (df.date.dt.day == d) & (df.country == 'Norway')
                                      for d in range(1, 10)}),
                        pd.DataFrame({f"s-jan{d}":
                                      (df.date.dt.month == 1) & (df.date.dt.day == d) & (df.country == 'Sweden')
                                      for d in range(1, 15)})
                       ],
                       axis=1)
    
    # May and June
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"may{d}":
                                      (df.date.dt.month == 5) & (df.date.dt.day == d) 
                                      for d in list(range(1, 10))}),
#                         pd.DataFrame({f"f-may{d}":
#                                       (df.date.dt.month == 5) & (df.date.dt.day == d) & (df.country == 'Finland') # end of the war
#                                       for d in [9]}),
                        pd.DataFrame({f"may{d}":
                                      (df.date.dt.month == 5) & (df.date.dt.day == d) & (df.country == 'Norway')
                                     for d in list(range(18, 26)) + [27]}),
                        pd.DataFrame({f"june{d}":
                                      (df.date.dt.month == 6) & (df.date.dt.day == d) & (df.country == 'Sweden')
                                      for d in list(range(8, 15))}),
                       ],
                       axis=1)
    
    # Last Wednesday of June
    wed_june_date = df.date.dt.year.map({2015: pd.Timestamp(('2015-06-24')),
                                         2016: pd.Timestamp(('2016-06-29')),
                                         2017: pd.Timestamp(('2017-06-28')),
                                         2018: pd.Timestamp(('2018-06-27')),
                                         2019: pd.Timestamp(('2019-06-26'))})
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"wed_june{d}": 
                                      (df.date - wed_june_date == np.timedelta64(d, "D")) & (df.country != 'Norway')
                                      for d in list(range(-4, 5))})],
                       axis=1)
    
    # First Sunday of November
    sun_nov_date = df.date.dt.year.map({2015: pd.Timestamp(('2015-11-1')),
                                         2016: pd.Timestamp(('2016-11-6')),
                                         2017: pd.Timestamp(('2017-11-5')),
                                         2018: pd.Timestamp(('2018-11-4')),
                                         2019: pd.Timestamp(('2019-11-3'))})
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"sun_nov{d}": 
                                      (df.date - sun_nov_date == np.timedelta64(d, "D")) & (df.country != 'Norway')
                                      for d in list(range(0, 9))})],
                       axis=1)
    
    # First half of December (Independence Day of Finland, 6th of December)
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"dec{d}":
                                      (df.date.dt.month == 12) & (df.date.dt.day == d) & (df.country == 'Finland')
                                      for d in list(range(6, 15))})],
                       axis=1)

    # Easter
    easter_date = df.date.apply(lambda date: pd.Timestamp(easter.easter(date.year)))
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"easter{d}": 
                                      (df.date - easter_date == np.timedelta64(d, "D"))
                                      for d in list(range(-2, 11)) + list(range(40, 48)) + list(range(51, 58))}),
                        pd.DataFrame({f"n_easter{d}": 
                                      (df.date - easter_date == np.timedelta64(d, "D")) & (df.country == 'Norway')
                                      for d in list(range(-3, 8)) + list(range(50, 61))})],
                       axis=1)
    
    return new_df.astype(np.float32)

train_df = engineer(original_train_df)
train_df['date'] = original_train_df.date
train_df['num_sold'] = original_train_df.num_sold.astype(np.float32)
test_df = engineer(original_test_df)

features = list(test_df.columns)
if NO_STORE: 
    features = [feature for feature in features if feature not in ['Discount Stickers', 'Stickers for Less']]

print(list(features))
    

['gdp', 'wd4', 'wd56', 'Canada', 'Finland', 'Italy', 'Kenya', 'Norway', 'Holographic Goose', 'Kaggle', 'Kaggle Tiers', 'Kerneler', 'HG_sin1', 'HG_cos1', 'Kaggle_sin1', 'Kaggle_cos1', 'KT_sin1', 'KT_cos1', 'Kerneler_sin1', 'Kerneler_cos1', 'HG_sin2', 'HG_cos2', 'Kaggle_sin2', 'Kaggle_cos2', 'KT_sin2', 'KT_cos2', 'Kerneler_sin2', 'Kerneler_cos2', 'dec24', 'dec25', 'dec26', 'dec27', 'dec28', 'dec29', 'dec30', 'dec31', 'n-dec25', 'n-dec26', 'n-dec27', 'n-dec28', 'n-dec29', 'n-dec30', 'n-dec31', 'f-jan1', 'f-jan2', 'f-jan3', 'f-jan4', 'f-jan5', 'f-jan6', 'f-jan7', 'f-jan8', 'f-jan9', 'f-jan10', 'f-jan11', 'f-jan12', 'f-jan13', 'f-jan14', 'n-jan1', 'n-jan2', 'n-jan3', 'n-jan4', 'n-jan5', 'n-jan6', 'n-jan7', 'n-jan8', 'n-jan9', 's-jan1', 's-jan2', 's-jan3', 's-jan4', 's-jan5', 's-jan6', 's-jan7', 's-jan8', 's-jan9', 's-jan10', 's-jan11', 's-jan12', 's-jan13', 's-jan14', 'may1', 'may2', 'may3', 'may4', 'may5', 'may6', 'may7', 'may8', 'may9', 'may18', 'may19', 'may20', 'may21', 'may22', 'may23'

In [11]:
train_df

,gdp,wd4,wd56,Canada,Finland,Italy,Kenya,Norway,Discount Stickers,Stickers for Less,...,n_easter53,n_easter54,n_easter55,n_easter56,n_easter57,n_easter58,n_easter59,n_easter60,date,num_sold
0,10.769792,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2010-01-01,NaN
1,10.769792,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2010-01-01,973.0
2,10.769792,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2010-01-01,906.0
3,10.769792,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2010-01-01,423.0
4,10.769792,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2010-01-01,491.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230125,10.948139,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-12-31,466.0
230126,10.948139,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-12-31,2907.0
230127,10.948139,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-12-31,2299.0
230128,10.948139,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-12-31,1242.0
